# ! WARNING !
# DO NOT CHANGE ANYTHING IN THIS APPLICATION EXCEPT FOR SECTIONS IN WHICH THIS IS EXPRESSLY REQUESTED BY THE DEVELOPER!
# IN THE EVENT OF AN ACCIDENT OF THIS KIND, USE THE BACKUP VERSION PROVIDED BY THE DEVELOPER OR ADDRESS HIM!

# 1. Install required packages
# All these dependencies are in an adjacent file called 'dependencies.bat' and their installation will be done by running the command from the next cell
# The development environment (IDE) must be launched in administrator mode to have necessary privileges to run the following commands

# Python library which is absolutely helpful for medical image analysis
!pip install SimpleITK # For reading and showing medical images

# Pre-processing
!pip install nipype # For pre-processing
!pip install opencv-python

# Data visualisation
!pip install ipywidgets
!pip install matplotlib
!pip install scikit-image

# Data augmentation
!pip install protobuf==3.19.0
!pip install tensorflow==2.2.0
!pip install keras==2.4.3

# Convolutional Neural Network (CNN)
!pip install torch
!pip install tensorflow-gpu==2.2.0

# Display statistics: ROC curve
!pip install -U scikit-learn

# DO NOT CHANGE LIBRARY VERSIONS EVEN IF NEWER VERSIONS HAVE APPEARED BECAUSE PROCESSING COMPATIBILY WILL BE LOST!
# THE USER MUST ASSUME RESPONSABILITY FOR ANY KIND OF CHANGE MADE

# This command should be executed only once in terminal, not every time the application is run
# The path must be customized according to the location of the script that contains the specified commands according to the existing one
COMMAND: cmd /c "d:/ATNN/BrainTumorDetector/dependencies.bat"

# 1'.In the unlikely event of a warning regarding the latest pip version, it is recommended to use COMMAND PROMPT to run the command in administrator mode:
python -m pip install --upgrade pip

# After that, run the command from the previous cell again

# 2. Import required libraries

In [1]:
# Data extraction
import zipfile # For file extraction
import glob # For retrieving files/pathnames matching a specified pattern
import re # Python has a module named re to work with RegEx - specifies a set of strings that matches it
import SimpleITK as sitk # Important package for medical imaging
import numpy as np
import ipywidgets as widgets
from IPython.display import display
from ipywidgets import interact # For interactive fields that allow viewing images
import matplotlib.pyplot as plt

#Pre-processing
from skimage.exposure import histogram
import cv2 # Library of programming functions mainly aimed at real-time computer vision
from numpy import expand_dims

# Data Augmentation
import tensorflow as tf # Open-source software library for machine learning and artificial intelligence 
from keras.preprocessing.image import ImageDataGenerator # ImageDataGenerator it's part of keras library that I need for data augmentation
import random # For generating random values
import skimage # Open-source image processing library
from skimage import exposure
import os # os module in Python provides functions for interacting with the operating system

# CNN
import torch # Library used for applications such as computer vision and natural language processing
import torch.nn as nn
import torch.nn.functional as F

from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation,Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.optimizers import Adam

# Display statistics
from cProfile import label
from turtle import color
from sklearn import metrics

# I. IMPORT DATASETS

# 3. Extract datasets

In [2]:
# My dataset is a zip file, therefore I have to extract it
def extract_dataset(addr, folder_name):
    dataset_path = addr
    # zipfile = module & ZipFile = The class for reading and writing ZIP files
    zfile = zipfile.ZipFile(dataset_path) # Open ZIP file at the indicated address
    zfile.extractall(folder_name) # Extract the files and store them in the mentioned folder

In [3]:
# Extracting the training data set
# The path must be customized according to the location of the training dataset
extract_dataset('D:/ATNN/BrainTumorDetector/MICCAI_BraTS2020_TrainingData.zip', 'DataSet/Training')

In [4]:
# Extracting the validation data set
# The path must be customized according to the location of the validation dataset
extract_dataset('D:/ATNN/BrainTumorDetector/MICCAI_BraTS2020_ValidationData.zip', 'DataSet/Validation')

In [ ]:
# Training paths
# The paths must be customized according to the location of the training dataset
# Array with paths (addresses) of all t1 type images
t1 = sorted(glob.glob('D:\\ATNN\\BrainTumorDetector\\DataSet\\Training\\MICCAI_BraTS2020_TrainingData\\*\\*t1.nii.gz'))
# Array with paths (addresses) of all t2 type images
t2 = sorted(glob.glob('D:\\ATNN\\BrainTumorDetector\\DataSet\\Training\\MICCAI_BraTS2020_TrainingData\\*\\*t2.nii.gz'))  
# Array with paths (addresses) of all flair type images
flair = sorted(glob.glob('D:\\ATNN\\BrainTumorDetector\\DataSet\\Training\\MICCAI_BraTS2020_TrainingData\\*\\*flair.nii.gz'))  
# Array with paths (addresses) of all t1ce type images
t1ce = sorted(glob.glob('D:\\ATNN\\BrainTumorDetector\\DataSet\\Training\\MICCAI_BraTS2020_TrainingData\\*\\*t1ce.nii.gz'))  
# Array with paths (addresses) of all seg type images
seg = sorted(glob.glob('D:\\ATNN\\BrainTumorDetector\\DataSet\\Training\\MICCAI_BraTS2020_TrainingData\\*\\*seg.nii.gz'))  

# Python's re.compile() method is used to compile a regular expression pattern provided as a string into a regex pattern object (re. Pattern)
# Later we can use this pattern object to search for a match inside different target strings using regex methods such as re
pattern = re.compile('D:\\\\ATNN\\\\BrainTumorDetector\\\\DataSet\\\\Training\\\\.*_(\w*)\.nii\.gz')

data_paths = [{
    pattern.findall(item)[0]: item for item in items # items is an array and item is an element of 'items' array 
}
    for items in list(zip(t1, t2, t1ce, flair, seg))] # list is a list of arrays

print('Number of training examples:', len(data_paths))
print('The format of an element of the training data set in its initial form:\n', data_paths[0])

In [ ]:
# Validation paths
# The paths must be customized according to the location of the training dataset
# Array with paths (addresses) of all t1 type images
t1_v = sorted(glob.glob('D:\\ATNN\\BrainTumorDetector\\DataSet\\Validation\\MICCAI_BraTS2020_ValidationData\\*\\*t1.nii.gz'))
# Array with paths (addresses) of all t2 type images
t2_v = sorted(glob.glob('D:\\ATNN\\BrainTumorDetector\\DataSet\\Validation\\MICCAI_BraTS2020_ValidationData\\*\\*t2.nii.gz'))  
# Array with paths (addresses) of all flair type images
flair_v = sorted(glob.glob('D:\\ATNN\\BrainTumorDetector\\DataSet\\Validation\\MICCAI_BraTS2020_ValidationData\\*\\*flair.nii.gz'))  
# Array with paths (addresses) of all t1ce type images
t1ce_v = sorted(glob.glob('D:\\ATNN\\BrainTumorDetector\\DataSet\\Validation\\MICCAI_BraTS2020_ValidationData\\*\\*t1ce.nii.gz'))  

# Python's re.compile() method is used to compile a regular expression pattern provided as a string into a regex pattern object (re. Pattern)
# Later we can use this pattern object to search for a match inside different target strings using regex methods such as re
pattern_v = re.compile('D:\\\\ATNN\\\\BrainTumorDetector\\\\DataSet\\\\Validation\\\\.*_(\w*)\.nii\.gz')

data_paths_v = [{
    pattern_v.findall(item_v)[0]: item_v for item_v in items_v # items is an array and item is an element of 'items' array 
}
    for items_v in list(zip(t1_v, t2_v, t1ce_v, flair_v))] # list is a list of arrays

print('Number of validation examples:', len(data_paths_v))
print('The format of an element of the validation data set in its initial form:\n', data_paths_v[0])

# 4. Read images

In [4]:
# Read images using SimpleITK
# read_img_sitk(img_path) returns an sitk image (<class 'SimpleITK.SimpleITK.Image'>)
def read_img_sitk(img_path):
    image_data = sitk.ReadImage(img_path)
    return image_data

In [ ]:
# Check the shape of images
# sitk images have their own characteristics which help in pre-processing and data augmentation
sitk_img = read_img_sitk(data_paths[0]['flair'])
sitk_shape = sitk_img.GetSize()
print('Shape of sitk_img: ', sitk_shape)

# 5. Conversion between SimpleITK and numpy array
# In order to give an image to a neural network, I should give it as a numpy array
# ATENTION! In this conversion, the order of dimensions change

In [ ]:
np_img = sitk.GetArrayFromImage(sitk_img)

# Check its shape
np_img.shape

# 6. Visualize samples

# 6.1 Training dataset

In [ ]:
# I want to have an interactive view which can be controlled by these parameters:
# 1. 369 patients
# 2. Four different modalities
# 3. Three different views
# 4. layer - which is chosen according to this dimension

@interact
def explore_3dimage_training(
    patient = widgets.IntSlider(
        min = 1,
        max = len(data_paths),
        step = 1,
        description='Pacient: ',
        value = 0),
                    
    layer = widgets.IntSlider(
        min = 1,
        max = 155,
        step = 1,
        description='Layer: ',
        value = 90),

    modality = widgets.ToggleButtons(
        value = 'flair',
        description='Modality: ',
        options = ['t1', 't2', 't1ce', 'flair', 'seg'],
        tooltips = ['t1', 't2', 't1ce', 'flair', 'seg'],
        button_style = 'primary'),

    view = widgets.ToggleButtons(
        value = 'axial',
        description='View: ',
        options = ['axial', 'sagittal', 'coronal'],
        tooltips = ['axial', 'sagittal', 'coronal'],
        button_style = 'primary')):

    if modality == 't1':
        modal = 't1'
    elif modality == 't2':
        modal = 't2'
    elif modality == 't1ce':
        modal = 't1ce'
    elif modality == 'flair':
        modal = 'flair'
    elif modality == 'seg':
        modal = 'seg'
    else:
        print('Error modality!')
    
    # Read image using sitk
    image = read_img_sitk(data_paths[patient - 1][modal])
    array_view = sitk.GetArrayViewFromImage(image) # Use this array for visualization

    if view == 'axial':
        array_view = array_view[layer - 1, :, :]
        plt.figure(figsize=(5,5))
    elif view == 'coronal':
        array_view = array_view[:, layer - 1, :]
        plt.figure(figsize=(8,8))
    elif view == 'sagittal':
        array_view = array_view[:, :, layer - 1]
        plt.figure(figsize=(8,8))
    else:
        print('Error view!')
    
    dim1 = str(array_view.shape[0])
    dim2 = str(array_view.shape[1])
    
    intensity1 = str(round(array_view.min(), 1))
    intensity2 = str(round(array_view.max(), 1))

    pacient = str(patient)
    layer = str(layer)
    modal = str(modal)
    view = str(view)

    plt.imshow(array_view, cmap='gray')
    plt.title('Explore Layers of Brain' + "\nImage size: " + dim1 + 'x' + dim2 + "\nIntensity: (" + intensity1 + ', ' + intensity2 + ')' + 
               "\nPacient: " + pacient  + "\nLayer: " + layer + "\nModality: " + modal + "\nView: " + view)
    plt.axis('off')

# 6.2 Validation dataset

In [ ]:
# I want to have an interactive view which can be controlled by these parameters:
# 1. 125 patients
# 2. Four different modalities
# 3. Three different views
# 4. layer - which is chosen according to this dimension

@interact
def explore_3dimage_validation(
    patient = widgets.IntSlider(
        min = 1,
        max = len(data_paths_v),
        step = 1,
        description='Pacient: ',
        value = 0),
                    
    layer = widgets.IntSlider(
        min = 1,
        max = 155,
        step = 1,
        description='Layer: ',
        value = 90),

    modality = widgets.ToggleButtons(
        value = 'flair',
        description='Modality: ',
        options = ['t1', 't2', 't1ce', 'flair'],
        tooltips = ['t1', 't2', 't1ce', 'flair'],
        button_style = 'primary'),

    view = widgets.ToggleButtons(
        value = 'axial',
        description='View: ',
        options = ['axial', 'sagittal', 'coronal'],
        tooltips = ['axial', 'sagittal', 'coronal'],
        button_style = 'primary')):

    if modality == 't1':
        modal = 't1'
    elif modality == 't2':
        modal = 't2'
    elif modality == 't1ce':
        modal = 't1ce'
    elif modality == 'flair':
        modal = 'flair'
    else:
        print('Error modality!')
    
    # Read image using sitk
    image = read_img_sitk(data_paths_v[patient - 1][modal])
    array_view = sitk.GetArrayViewFromImage(image) # Use this array for visualization

    if view == 'axial':
        array_view = array_view[layer - 1, :, :]
        plt.figure(figsize=(6.5,6.5))
    elif view == 'coronal':
        array_view = array_view[:, layer - 1, :]
        plt.figure(figsize=(8,8))
    elif view == 'sagittal':
        array_view = array_view[:, :, layer - 1]
        plt.figure(figsize=(8,8))
    else:
        print('Error view!')
    
    dim1 = str(array_view.shape[0])
    dim2 = str(array_view.shape[1])
    
    intensity1 = str(round(array_view.min(),1))
    intensity2 = str(round(array_view.max(),1))

    pacient = str(patient)
    layer = str(layer)
    modal = str(modal)
    view = str(view)

    plt.imshow(array_view, cmap='gray')
    plt.title('Explore Layers of Brain' + "\nImage size: " + dim1 + 'x' + dim2 + "\nIntensity: (" + intensity1 + ', ' + intensity2 + ')' + 
                "\nPacient: " + pacient + "\nLayer: " + layer + "\nModality: " + modal + "\nView: " + view)

# 7. Component selection for training dataset

In [7]:
# From the entire dataset, I select within only the FLAIR components and the SEG components that will be used in the further processing (training)
# Only the FLAIR modality is chosen for each patient because through it the whole tumor can be segmented
noInstances = len(data_paths) # Number of pacients OR number of training instances
images = [read_img_sitk(data_paths[i]['flair']) for i in range(noInstances)]
segmented = [read_img_sitk(data_paths[i]['seg']) for i in range(noInstances)]

# 8. Select axial view only - I select all the slices for each pacient, the images being from the axial perspective of visualization

# Displaying slices of interest from the initial dataset. The reason why these slices are of interest is simple. Slice 90 highlights the most numerous details regarding a possible tumor (representing the slice in the area with the largest diameter of the brain). Slices 80 and 100 are close to 90 and these are experimentally selected to determine if the results of the training process can be improved in this way.

In [9]:
noLayers = 155
trainingImages_beforeCropping = [sitk.GetArrayViewFromImage(image)[layer, :, :] for image in images for layer in range(noLayers)]
trainingSegmented_beforeCropping = [sitk.GetArrayViewFromImage(segment)[layer, :, :] for segment in segmented for layer in range(noLayers)]

In [ ]:
# I want to have an interactive view which can be controlled by this parameter:
# 57195 images

@interact
def explore_2dimages_training(
    image = widgets.IntSlider(
        min = 1,
        max = len(trainingImages_beforeCropping),
        step = 1,
        description='Image: ',
        value = 49070)):
    
    index = image - 1
    array_view = trainingImages_beforeCropping[index] # Use this array for visualization

    dim1 = str(array_view.shape[0])
    dim2 = str(array_view.shape[1])
    
    intensity1 = str(round(array_view.min(), 1))
    intensity2 = str(round(array_view.max(), 1))

    pacient = str(int(index/155) + 1)
    layer = str(int(index%155) + 1)
    modal = str('FLAIR')
    view = str('axial')

    plt.figure(figsize=(5,5))
    plt.imshow(array_view, cmap = 'gray')
    plt.title('Explore images from dataset' + "\nImage size: " + dim1 + 'x' + dim2 + "\nIntensity: (" + intensity1 + ', ' + intensity2 + ')' +
            "\nPacient: " + pacient + "\nLayer: " + layer + "\nModality: " + modal + "\nView: " + view)

# In order to be able to find the best results in term of accuracy, I will create several training datasets, starting from the initial dataset, considering the slices presented above

# 9. Crop images - Before selecting the datasets from the initial dataset, the component elements will be cut

In [11]:
# The value of dimensions were not chosen arbitrarily, but are indicated by the documentation of the 'Brain Tumor Segmentation Based on Deep
# Learning’s Feature Representation' study
# These values are:
# 192 for height
# 152 for width
def crop_images(img):
    # Region Of Interest
    start_x = int(int(img.shape[0] - 192) / 2)
    end_x = start_x + 192
    start_y = int(int(img.shape[1] - 152) / 2)
    end_y = start_y + 152
    
    cropped_img = img [start_x:end_x,
                       start_y:end_y]
    
    return cropped_img

# All images comply with a standard size within the BraTS_2020 dataset and thus it is guaranteed that no problems may occur when cropping

In [12]:
# The crop function is applied to the entire training dataset
cropped_images = [crop_images(trainingImages_beforeCropping[i]) for i in range(len(trainingImages_beforeCropping))]
cropped_segmented = [crop_images(trainingSegmented_beforeCropping[i]) for i in range(len(trainingImages_beforeCropping))]

In [ ]:
# I want to have an interactive view which can be controlled by this parameter:
# 57195 images

@interact
def explore_2d_cropped_images_training(
    image = widgets.IntSlider(
        min = 1,
        max = len(trainingImages_beforeCropping),
        step = 1,
        description='Image: ',
        value = 49070)):
    
    index = image - 1
    array_view_1 = trainingImages_beforeCropping[index] # Use this array for visualization
    array_view_2 = cropped_images[index]
    
    # DImension of images
    dim1_1 = str(array_view_1.shape[0])
    dim1_2 = str(array_view_1.shape[1])
    dim2_1 = str(array_view_2.shape[0])
    dim2_2 = str(array_view_2.shape[1])
    
    # Intensity ranges for images
    intensity1_1 = str(round(array_view_1.min(), 1)) 
    intensity1_2 = str(round(array_view_1.max(), 1))
    intensity2_1 = str(round(array_view_2.min(), 1))
    intensity2_2 = str(round(array_view_2.max(), 1))

    pacient = str(int(index/155) + 1)
    layer = str(int(index%155) + 1)
    modal = str('FLAIR')
    view = str('axial')

    # Histograms
    img_before = trainingImages_beforeCropping[index]
    img_after = cropped_images[index]

    hist1_1, hist_centers1_1 = histogram(img_before, nbins=256)
    img_cdf1_1, bins1_1 = exposure.cumulative_distribution(img_before)

    hist2_1, hist_centers2_1 = histogram(img_after, nbins=256)
    img_cdf2_1, bins2_1 = exposure.cumulative_distribution(img_after)

    fig, ax = plt.subplots(2,2, figsize=(10,8.5)) # Set dimensions for view field
    ax[0,0].set_title('Before cropping\n Image size: ' + dim1_1 + 'x' + dim1_2 + '\nIntensity: (' + intensity1_1 + ', ' + intensity1_2 + ')' +
                       '\nPacient: ' + pacient + '\nLayer: ' + layer + '\nModality: ' + modal + '\nView: ' + view) 
    ax[0,0].imshow(array_view_1, cmap = 'gray')
    
    ax[0,1].set_title('After cropping\n Image size: ' + dim2_1 + 'x' + dim2_2 + '\nIntensity: (' + intensity2_1 + ', ' + intensity2_2 + ')' +
                       '\nPacient: ' + pacient + '\nLayer: ' + layer + '\nModality: ' + modal + '\nView: ' + view)
    ax[0,1].imshow(array_view_2, cmap = 'gray')


    ax[1,0].plot(bins1_1, img_cdf1_1, 'r')
    ax[1,0].plot(hist_centers1_1, hist1_1)
    
    ax[1,1].plot(bins2_1, img_cdf2_1, 'r')
    ax[1,1].plot(hist_centers2_1, hist2_1)

    # Zooming for much clearer plots beacause images have different scaling
    ax[1,0].axis([0.01, 600, 0, 3e2])
    ax[1,1].axis([0.01, 800, 0, 3e2])

# As can be seen in the cell above, I have cropped the image successfully and now the image is perfectly framed in the quadrant and most of the black pixels have been removed
# As can be seen and as expected, the histograms of the images do not change through the cropping process

# 10. Creating datasets

In [13]:
# 1st - the first dataset created brings together only 369 elements. For each patient there are 155 slices in the dataset after the crop
# operation was performed. Each element in the new dataset represents the 90 slice corresponding to each patient
images_dataset_1 = [cropped_images[i * 155 + 90] for i in range(len(images))]
segmented_dataset_1 = [cropped_segmented[i * 155 + 90] for i in range(len(images))]


# 2nd - the second dataset created brings together 369x2=738 elements. For each patient there are 155 slices in the dataset after the crop
# operation was performed. This dataset contains slices 80 and 90 of the dataset made after the crop operation
noFlatten_images_dataset_2 = [[cropped_images[i * 155 + 80], cropped_images[i * 155 + 90]]  for i in range(len(images))]
noFlatten_segmented_dataset_2 = [[cropped_segmented[i * 155 + 80], cropped_segmented[i * 155 + 90]] for i in range(len(images))]

# Flatten the list of lists obtained
images_dataset_2 = [item for sublist in noFlatten_images_dataset_2 for item in sublist]
segmented_dataset_2 = [item for sublist in noFlatten_segmented_dataset_2 for item in sublist]


# 3rd - the third dataset created brings together 369x3=1107 elements. For each patient there are 155 slices in the dataset after the crop
# operation was performed. This dataset contains slices 80, 90 and 100 of the dataset made after the crop operation
noFlatten_images_dataset_3 = [[cropped_images[i * 155 + 80], cropped_images[i * 155 + 90], cropped_images[i * 155 + 100]]  for i in range(len(images))]
noFlatten_segmented_dataset_3 = [[cropped_segmented[i * 155 + 80], cropped_segmented[i * 155 + 90],cropped_segmented[i * 155 + 100]] for i in range(len(images))]

# Flatten the list of lists obtained
images_dataset_3 = [item for sublist in noFlatten_images_dataset_3 for item in sublist]
segmented_dataset_3 = [item for sublist in noFlatten_segmented_dataset_3 for item in sublist]

In [ ]:
list_1 = [i+1 for i in range(len(images_dataset_1))]
list_2 = [i+1 for i in range(len(images_dataset_2))]
list_3 = [i+1 for i in range(len(images_dataset_3))]

datasets = {'Dataset 1':list_1, 'Dataset 2':list_2, 'Dataset 3':list_3}
pacients = {key: widgets.Select(options = datasets[key]) for key in datasets}

def get_dataset_and_pacient():
    if i.children[0].value == 'Dataset 1':
        images_dataset = images_dataset_1
    if i.children[0].value == 'Dataset 2':
        images_dataset = images_dataset_2
    if i.children[0].value == 'Dataset 3':
        images_dataset = images_dataset_3

    plt.imshow(images_dataset[i.children[1].value - 1], cmap = 'gray')

def print_pacient(**func_kwargs):
    get_dataset_and_pacient()

def select_dataset(datasetNo):
    new_i = widgets.interactive(print_pacient, datasetNo=datasetSelected, pacientNo=pacients[datasetNo['new']])
    i.children = new_i.children

datasetSelected = widgets.Select(options = list(datasets.keys()))
init = datasetSelected.value
pacientSelected = pacients[init]

datasetSelected.observe(select_dataset, 'value')

i = widgets.interactive(print_pacient, datasetNo=datasetSelected, pacientNo = pacientSelected)

display(i)

# 11. Create binary labels for each dataset

In [15]:
# In order to create binary labels for images in the training datasets, check that the segmented images contain at least one non-zero pixel value
# npnonzero function returns the indices of the elements that are non-zero
# Function returns a tuple of arrays, one for each dimension of ARGUMENT, containing the indices of the non-zero elements in that dimension. The
# values in ARGUMENT are always tested and returned in row-major, C-style order

labels_1 = [np.nonzero(segmented_dataset_1[i]) for i in range(len(segmented_dataset_1))]
labels_2 = [np.nonzero(segmented_dataset_2[i]) for i in range(len(segmented_dataset_2))]
labels_3 = [np.nonzero(segmented_dataset_3[i]) for i in range(len(segmented_dataset_3))]

trainingLabels_1 = [ 1 if len((labels_1[i])[0]) > 0 and len((labels_1[i])[1]) > 0 else 0 for i in range(len(labels_1))]
trainingLabels_2 = [ 1 if len((labels_2[i])[0]) > 0 and len((labels_2[i])[1]) > 0 else 0 for i in range(len(labels_2))]
trainingLabels_3 = [ 1 if len((labels_3[i])[0]) > 0 and len((labels_3[i])[1]) > 0 else 0 for i in range(len(labels_3))]

# In order to be able to use a dataset in the augmentation process it is necessary to specify in its shape the number of color channels
# for each instance in the dataset, not only the values for length and width

trainingImages_1 = [expand_dims(images_dataset_1[i], axis=2) for i in range(len(images_dataset_1))]
trainingImages_2 = [expand_dims(images_dataset_2[i], axis=2) for i in range(len(images_dataset_2))]
trainingImages_3 = [expand_dims(images_dataset_3[i], axis=2) for i in range(len(images_dataset_3))]

trainingImages_1 = [trainingImages_1[i].reshape((1, ) + trainingImages_1[i].shape) for i in range(len(trainingImages_1))]
trainingImages_2 = [trainingImages_2[i].reshape((1, ) + trainingImages_2[i].shape) for i in range(len(trainingImages_2))]
trainingImages_3 = [trainingImages_3[i].reshape((1, ) + trainingImages_3[i].shape) for i in range(len(trainingImages_3))]

# II. DATA AUGMENTATION

# I will create 3 ways to generate new datasets based on existing datasets. This means that from 3 existing datasets will result 9 training datasets

# 12. Creating auxiliary functions for image generators

In [16]:
# Add random noise to an image
def add_random_noise(img):
    VARIABILITY = 1.5
    deviation = VARIABILITY*random.random()
    while (deviation < 0.15 or deviation > 0.45):
        deviation = VARIABILITY*random.random()
        
    mask_1 = np.where(img == 0)

    modified = np.copy(img)
    modified = modified.astype(np.float32)

    noise = np.random.normal(0, (deviation*modified.mean())/(1.5), modified.shape)
    noise = noise.astype(np.float32)

    modified += noise

    modified[mask_1] = modified.min()

    modified = modified/(modified.max()/255.0)

    mask_2 = np.where(modified < 0)
    modified[mask_2] = 0

    mask_3 = np.where(modified > 255)
    modified[mask_3] = 255

    return modified

In [17]:
# Add Gaussian blur
def add_Gaussian_blur(image):
    sigma = 0.7
    blurred = skimage.filters.gaussian(
        image, sigma=(sigma, sigma), truncate=3.5, channel_axis=True)
    return blurred

# 13.1 First type of data generator - Construct an instance of the ImageDataGenerator class for each type of augmenting

In [18]:
# Flip horizontally
datagen_flip_horizontal = ImageDataGenerator(
    rescale=1./255,
    horizontal_flip=True)

# Flip vertically
datagen_flip_vertical = ImageDataGenerator(
    rescale=1./255,
    vertical_flip=True)

# Rotation_plus
datagen_rotation_plus = ImageDataGenerator(
    rescale=1./255,
    rotation_range=90)

# Rotation_minus
datagen_rotation_minus = ImageDataGenerator(
    rescale=1./255,
    rotation_range=-90)

# Shift horizontal & vertical
datagen_shift = ImageDataGenerator(
    rescale=1./255,
    height_shift_range=[-20,20],
    width_shift_range=[-20,20])

# Noise addition
datagen_noise = ImageDataGenerator(
    rescale=1./255,
    preprocessing_function=add_random_noise,
    fill_mode='constant')

# Blur image
datagen_blur = ImageDataGenerator(
    rescale=1./255,
    preprocessing_function=add_Gaussian_blur)

# Modify brightness of image
datagen_brightness = ImageDataGenerator(
    brightness_range=(0.2, 0.8))

# Shear image
datagen_shear = ImageDataGenerator(
    shear_range=0.2)

# Zoom image
datagen_zoom = ImageDataGenerator(
    zoom_range=[0.7,1.0])

# 1/9 training dataset

In [19]:
# Generate batches of augmented images for each image from initial dataset
datagen_flip_horizontal_batch_1 = [datagen_flip_horizontal.flow(trainingImages_1[i]) for i in range(len(trainingImages_1))]
datagen_flip_vertical_batch_1 = [datagen_flip_vertical.flow(trainingImages_1[i]) for i in range(len(trainingImages_1))]
datagen_rotation_plus_batch_1 = [datagen_rotation_plus.flow(trainingImages_1[i]) for i in range(len(trainingImages_1))]
datagen_rotation_minus_batch_1 = [datagen_rotation_minus.flow(trainingImages_1[i]) for i in range(len(trainingImages_1))]
datagen_shift_batch_1 = [datagen_shift.flow(trainingImages_1[i]) for i in range(len(trainingImages_1))]
datagen_noise_batch_1 = [datagen_noise.flow(trainingImages_1[i]) for i in range(len(trainingImages_1))]
datagen_blur_batch_1 = [datagen_blur.flow(trainingImages_1[i]) for i in range(len(trainingImages_1))]
datagen_brightness_batch_1 = [datagen_brightness.flow(trainingImages_1[i]) for i in range(len(trainingImages_1))]
datagen_shear_batch_1 = [datagen_shear.flow(trainingImages_1[i]) for i in range(len(trainingImages_1))]
datagen_zoom_batch_1 = [datagen_zoom.flow(trainingImages_1[i]) for i in range(len(trainingImages_1))]


# Get 5 samples of each type of augmented images for each initial image
notFlatten_augmeneted_datagen_flip_horizontal_1 = [[next(item)[0].astype(np.float32) for i in range(5)] for item in datagen_flip_horizontal_batch_1]
notFlatten_augmented_datagen_flip_vertical_1 = [[next(item)[0].astype(np.float32) for i in range(5)] for item in datagen_flip_vertical_batch_1]
notFlatten_augmented_datagen_rotation_plus_1 = [[next(item)[0].astype(np.float32) for i in range(5)] for item in datagen_rotation_plus_batch_1]
notFlatten_augmented_datagen_rotation_minus_1 = [[next(item)[0].astype(np.float32) for i in range(5)] for item in datagen_rotation_minus_batch_1]
notFlatten_augmented_datagen_shift_1 = [[next(item)[0].astype(np.float32) for i in range(5)] for item in datagen_shift_batch_1]
notFlatten_augmented_datagen_noise_1 = [[next(item)[0].astype(np.float32) for i in range(5)] for item in datagen_noise_batch_1]
notFlatten_augmented_datagen_blur_1 = [[next(item)[0].astype(np.float32) for i in range(5)] for item in datagen_blur_batch_1]
notFlatten_augmented_datagen_brightness_1 = [[next(item)[0].astype(np.float32) for i in range(5)] for item in datagen_brightness_batch_1]
notFlatten_augmented_datagen_shear_1 = [[next(item)[0].astype(np.float32) for i in range(5)] for item in datagen_shear_batch_1]
notFlatten_augmented_datagen_zoom_1 = [[next(item)[0].astype(np.float32) for i in range(5)] for item in datagen_zoom_batch_1]


# Flatten lists
augmeneted_datagen_flip_horizontal_1 = [item for sublist in notFlatten_augmeneted_datagen_flip_horizontal_1 for item in sublist]
augmented_datagen_flip_vertical_1 = [item for sublist in notFlatten_augmented_datagen_flip_vertical_1 for item in sublist]
augmented_datagen_rotation_plus_1 = [item for sublist in notFlatten_augmented_datagen_rotation_plus_1 for item in sublist]
augmented_datagen_rotation_minus_1 = [item for sublist in notFlatten_augmented_datagen_rotation_minus_1 for item in sublist]
augmented_datagen_shift_1 = [item for sublist in notFlatten_augmented_datagen_shift_1 for item in sublist]
augmented_datagen_noise_1 = [item for sublist in notFlatten_augmented_datagen_noise_1 for item in sublist]
augmented_datagen_blur_1 = [item for sublist in notFlatten_augmented_datagen_blur_1 for item in sublist]
augmented_datagen_brightness_1 = [item for sublist in notFlatten_augmented_datagen_brightness_1 for item in sublist]
augmented_datagen_shear_1 = [item for sublist in notFlatten_augmented_datagen_shear_1 for item in sublist]
augmented_datagen_zoom_1 = [item for sublist in notFlatten_augmented_datagen_zoom_1 for item in sublist]

In [20]:
# 1/9 training dataset
finalImages_1 = []
finalLabels_1 = []

for i in range(len(trainingImages_1)):
    finalImages_1.append(cv2.resize(trainingImages_1[i][0,:,:,0], dsize=(128, 128), interpolation=cv2.INTER_CUBIC))
    finalLabels_1.append(trainingLabels_1[i])

for i in range(len(augmeneted_datagen_flip_horizontal_1)):
    
    # Resize image in 128x128 format
    finalImages_1.append(cv2.resize(augmeneted_datagen_flip_horizontal_1[i][:,:,0], dsize=(128, 128), interpolation=cv2.INTER_CUBIC))
    finalLabels_1.append(trainingLabels_1[int(i/5)])

    finalImages_1.append(cv2.resize(augmented_datagen_flip_vertical_1[i][:,:,0], dsize=(128, 128), interpolation=cv2.INTER_CUBIC))
    finalLabels_1.append(trainingLabels_1[int(i/5)])

    finalImages_1.append(cv2.resize(augmented_datagen_rotation_plus_1[i][:,:,0], dsize=(128, 128), interpolation=cv2.INTER_CUBIC))
    finalLabels_1.append(trainingLabels_1[int(i/5)])

    finalImages_1.append(cv2.resize(augmented_datagen_rotation_minus_1[i][:,:,0], dsize=(128, 128), interpolation=cv2.INTER_CUBIC))
    finalLabels_1.append(trainingLabels_1[int(i/5)])

    finalImages_1.append(cv2.resize(augmented_datagen_shift_1[i][:,:,0], dsize=(128, 128), interpolation=cv2.INTER_CUBIC))
    finalLabels_1.append(trainingLabels_1[int(i/5)])

    finalImages_1.append(cv2.resize(augmented_datagen_noise_1[i][:,:,0], dsize=(128, 128), interpolation=cv2.INTER_CUBIC))
    finalLabels_1.append(trainingLabels_1[int(i/5)])

    finalImages_1.append(cv2.resize(augmented_datagen_blur_1[i][:,:,0], dsize=(128, 128), interpolation=cv2.INTER_CUBIC))
    finalLabels_1.append(trainingLabels_1[int(i/5)])

    finalImages_1.append(cv2.resize(augmented_datagen_brightness_1[i][:,:,0], dsize=(128, 128), interpolation=cv2.INTER_CUBIC))
    finalLabels_1.append(trainingLabels_1[int(i/5)])

    finalImages_1.append(cv2.resize(augmented_datagen_shear_1[i][:,:,0], dsize=(128, 128), interpolation=cv2.INTER_CUBIC))
    finalLabels_1.append(trainingLabels_1[int(i/5)])

    finalImages_1.append(cv2.resize(augmented_datagen_zoom_1[i][:,:,0], dsize=(128, 128), interpolation=cv2.INTER_CUBIC))
    finalLabels_1.append(trainingLabels_1[int(i/5)])

# Rescale intensity range
for i in range(len(finalImages_1)):
    mask_1 = np.where(finalImages_1[i] == 0)

    (finalImages_1[i])[mask_1] = (finalImages_1[i]).min()  

    finalImages_1[i] = (finalImages_1[i])/((finalImages_1[i]).max()/255.0)

    mask_2 = np.where(finalImages_1[i] < 0)
    (finalImages_1[i])[mask_2] = 0

    mask_3 = np.where(finalImages_1[i] > 255)
    (finalImages_1[i])[mask_3] = 255

# Augmented data visualization INTERFACE - interactive field for data visualization

In [ ]:
# Interactive field
# I want to have an interactive view which can be controlled by this parameter:
# 18819 images

@interact
def explore_final_training_dataset_1(
    image = widgets.IntSlider(
        min = 1,
        max = len(finalImages_1),
        step = 1,
        description='Image: ',
        value = 10000)):
    
    index = image - 1
    array_view = finalImages_1[index] # Use this array for visualization

    dim1 = str(array_view.shape[0])
    dim2 = str(array_view.shape[1])
    
    intensity1 = str(round(array_view.min(), 1))
    intensity2 = str(round(array_view.max(), 1))

    pacient = str(int(index/51) + 1)
    augmented_instance = str(int(index%51) + 1)

    modal = str('FLAIR')
    view = str('axial')

    plt.figure(figsize=(5,5))
    plt.title(label = 'Explore images from dataset' + "\nImage size: " + dim1 + 'x' + dim2 + "\nIntensity: (" + intensity1 + ', ' + intensity2 + ')' +
    "\nPacient: " + pacient + "\nLayer: 90" + "\nAugmented instance: " + augmented_instance + "\nModality: " + modal + "\nView: " + view)
    plt.imshow(array_view, cmap = 'gray')
    plt.axis('off')
    

# 2/9 & 3/9 training datasets

In [27]:
# 2/9
# Generate batches of augmented images from this image
datagen_flip_horizontal_batch_2 = [datagen_flip_horizontal.flow(trainingImages_2[i]) for i in range(len(trainingImages_2))]
datagen_flip_vertical_batch_2 = [datagen_flip_vertical.flow(trainingImages_2[i]) for i in range(len(trainingImages_2))]
datagen_rotation_plus_batch_2 = [datagen_rotation_plus.flow(trainingImages_2[i]) for i in range(len(trainingImages_2))]
datagen_rotation_minus_batch_2 = [datagen_rotation_minus.flow(trainingImages_2[i]) for i in range(len(trainingImages_2))]
datagen_shift_batch_2 = [datagen_shift.flow(trainingImages_2[i]) for i in range(len(trainingImages_2))]
datagen_noise_batch_2 = [datagen_noise.flow(trainingImages_2[i]) for i in range(len(trainingImages_2))]
datagen_blur_batch_2 = [datagen_blur.flow(trainingImages_2[i]) for i in range(len(trainingImages_2))]

# Get 2 samples of each type of augmented images for each initial image
notFlatten_augmeneted_datagen_flip_horizontal_2 = [[next(item)[0].astype(np.float32) for i in range(2)] for item in datagen_flip_horizontal_batch_2]
notFlatten_augmented_datagen_flip_vertical_2 = [[next(item)[0].astype(np.float32) for i in range(2)] for item in datagen_flip_vertical_batch_2]
notFlatten_augmented_datagen_rotation_plus_2 = [[next(item)[0].astype(np.float32) for i in range(2)] for item in datagen_rotation_plus_batch_2]
notFlatten_augmented_datagen_rotation_minus_2 = [[next(item)[0].astype(np.float32) for i in range(2)] for item in datagen_rotation_minus_batch_2]
notFlatten_augmented_datagen_shift_2 = [[next(item)[0].astype(np.float32) for i in range(2)] for item in datagen_shift_batch_2]
notFlatten_augmented_datagen_noise_2 = [[next(item)[0].astype(np.float32) for i in range(2)] for item in datagen_noise_batch_2]
notFlatten_augmented_datagen_blur_2 = [[next(item)[0].astype(np.float32) for i in range(2)] for item in datagen_blur_batch_2]

# Flatten lists
augmeneted_datagen_flip_horizontal_2 = [item for sublist in notFlatten_augmeneted_datagen_flip_horizontal_2 for item in sublist]
augmented_datagen_flip_vertical_2 = [item for sublist in notFlatten_augmented_datagen_flip_vertical_2 for item in sublist]
augmented_datagen_rotation_plus_2 = [item for sublist in notFlatten_augmented_datagen_rotation_plus_2 for item in sublist]
augmented_datagen_rotation_minus_2 = [item for sublist in notFlatten_augmented_datagen_rotation_minus_2 for item in sublist]
augmented_datagen_shift_2 = [item for sublist in notFlatten_augmented_datagen_shift_2 for item in sublist]
augmented_datagen_noise_2 = [item for sublist in notFlatten_augmented_datagen_noise_2 for item in sublist]
augmented_datagen_blur_2 = [item for sublist in notFlatten_augmented_datagen_blur_2 for item in sublist]

# 2/9 training dataset
finalImages_2 = []
finalLabels_2 = []

for i in range(len(trainingImages_2)):
    finalImages_2.append(cv2.resize(trainingImages_2[i][0,:,:,0], dsize=(128, 128), interpolation=cv2.INTER_CUBIC))
    finalLabels_2.append(trainingLabels_2[i])

for i in range(len(augmeneted_datagen_flip_horizontal_2)):
    
    # Resize image in 128x128 format
    finalImages_2.append(cv2.resize(augmeneted_datagen_flip_horizontal_2[i][:,:,0], dsize=(128, 128), interpolation=cv2.INTER_CUBIC))
    finalLabels_2.append(trainingLabels_2[int(i/2)])

    finalImages_2.append(cv2.resize(augmented_datagen_flip_vertical_2[i][:,:,0], dsize=(128, 128), interpolation=cv2.INTER_CUBIC))
    finalLabels_2.append(trainingLabels_2[int(i/2)])

    finalImages_2.append(cv2.resize(augmented_datagen_rotation_plus_2[i][:,:,0], dsize=(128, 128), interpolation=cv2.INTER_CUBIC))
    finalLabels_2.append(trainingLabels_2[int(i/2)])

    finalImages_2.append(cv2.resize(augmented_datagen_rotation_minus_2[i][:,:,0], dsize=(128, 128), interpolation=cv2.INTER_CUBIC))
    finalLabels_2.append(trainingLabels_2[int(i/2)])

    finalImages_2.append(cv2.resize(augmented_datagen_shift_2[i][:,:,0], dsize=(128, 128), interpolation=cv2.INTER_CUBIC))
    finalLabels_2.append(trainingLabels_2[int(i/2)])

    finalImages_2.append(cv2.resize(augmented_datagen_noise_2[i][:,:,0], dsize=(128, 128), interpolation=cv2.INTER_CUBIC))
    finalLabels_2.append(trainingLabels_2[int(i/2)])

    finalImages_2.append(cv2.resize(augmented_datagen_blur_2[i][:,:,0], dsize=(128, 128), interpolation=cv2.INTER_CUBIC))
    finalLabels_2.append(trainingLabels_2[int(i/2)])


# 3/9
# Generate batches of augmented images from this image
datagen_flip_horizontal_batch_3 = [datagen_flip_horizontal.flow(trainingImages_3[i]) for i in range(len(trainingImages_3))]
datagen_flip_vertical_batch_3 = [datagen_flip_vertical.flow(trainingImages_3[i]) for i in range(len(trainingImages_3))]
datagen_rotation_plus_batch_3 = [datagen_rotation_plus.flow(trainingImages_3[i]) for i in range(len(trainingImages_3))]
datagen_rotation_minus_batch_3 = [datagen_rotation_minus.flow(trainingImages_3[i]) for i in range(len(trainingImages_3))]
datagen_shift_batch_3 = [datagen_shift.flow(trainingImages_3[i]) for i in range(len(trainingImages_3))]
datagen_noise_batch_3 = [datagen_noise.flow(trainingImages_3[i]) for i in range(len(trainingImages_3))]
datagen_blur_batch_3 = [datagen_blur.flow(trainingImages_3[i]) for i in range(len(trainingImages_3))]

# Get 2 samples of each type of augmented images for each initial image
notFlatten_augmeneted_datagen_flip_horizontal_3 = [[next(item)[0].astype(np.float32) for i in range(2)] for item in datagen_flip_horizontal_batch_3]
notFlatten_augmented_datagen_flip_vertical_3 = [[next(item)[0].astype(np.float32) for i in range(2)] for item in datagen_flip_vertical_batch_3]
notFlatten_augmented_datagen_rotation_plus_3 = [[next(item)[0].astype(np.float32) for i in range(2)] for item in datagen_rotation_plus_batch_3]
notFlatten_augmented_datagen_rotation_minus_3 = [[next(item)[0].astype(np.float32) for i in range(2)] for item in datagen_rotation_minus_batch_3]
notFlatten_augmented_datagen_shift_3 = [[next(item)[0].astype(np.float32) for i in range(2)] for item in datagen_shift_batch_3]
notFlatten_augmented_datagen_noise_3 = [[next(item)[0].astype(np.float32) for i in range(2)] for item in datagen_noise_batch_3]
notFlatten_augmented_datagen_blur_3 = [[next(item)[0].astype(np.float32) for i in range(2)] for item in datagen_blur_batch_3]

# Flatten lists
augmeneted_datagen_flip_horizontal_3 = [item for sublist in notFlatten_augmeneted_datagen_flip_horizontal_3 for item in sublist]
augmented_datagen_flip_vertical_3 = [item for sublist in notFlatten_augmented_datagen_flip_vertical_3 for item in sublist]
augmented_datagen_rotation_plus_3 = [item for sublist in notFlatten_augmented_datagen_rotation_plus_3 for item in sublist]
augmented_datagen_rotation_minus_3 = [item for sublist in notFlatten_augmented_datagen_rotation_minus_3 for item in sublist]
augmented_datagen_shift_3 = [item for sublist in notFlatten_augmented_datagen_shift_3 for item in sublist]
augmented_datagen_noise_3 = [item for sublist in notFlatten_augmented_datagen_noise_3 for item in sublist]
augmented_datagen_blur_3 = [item for sublist in notFlatten_augmented_datagen_blur_3 for item in sublist]

# 3/9 training dataset
finalImages_3 = []
finalLabels_3 = []

for i in range(len(trainingImages_3)):
    finalImages_3.append(cv2.resize(trainingImages_3[i][0,:,:,0], dsize=(128, 128), interpolation=cv2.INTER_CUBIC))
    finalLabels_3.append(trainingLabels_3[i])

for i in range(len(augmeneted_datagen_flip_horizontal_3)):
    
    # Resize image in 128x128 format
    finalImages_3.append(cv2.resize(augmeneted_datagen_flip_horizontal_3[i][:,:,0], dsize=(128, 128), interpolation=cv2.INTER_CUBIC))
    finalLabels_3.append(trainingLabels_3[int(i/2)])

    finalImages_3.append(cv2.resize(augmented_datagen_flip_vertical_3[i][:,:,0], dsize=(128, 128), interpolation=cv2.INTER_CUBIC))
    finalLabels_3.append(trainingLabels_3[int(i/2)])

    finalImages_3.append(cv2.resize(augmented_datagen_rotation_plus_3[i][:,:,0], dsize=(128, 128), interpolation=cv2.INTER_CUBIC))
    finalLabels_3.append(trainingLabels_3[int(i/2)])

    finalImages_3.append(cv2.resize(augmented_datagen_rotation_minus_3[i][:,:,0], dsize=(128, 128), interpolation=cv2.INTER_CUBIC))
    finalLabels_3.append(trainingLabels_3[int(i/2)])

    finalImages_3.append(cv2.resize(augmented_datagen_shift_3[i][:,:,0], dsize=(128, 128), interpolation=cv2.INTER_CUBIC))
    finalLabels_3.append(trainingLabels_3[int(i/2)])

    finalImages_3.append(cv2.resize(augmented_datagen_noise_3[i][:,:,0], dsize=(128, 128), interpolation=cv2.INTER_CUBIC))
    finalLabels_3.append(trainingLabels_3[int(i/2)])

    finalImages_3.append(cv2.resize(augmented_datagen_blur_3[i][:,:,0], dsize=(128, 128), interpolation=cv2.INTER_CUBIC))
    finalLabels_3.append(trainingLabels_3[int(i/2)])

# 13.2 Second type of data generator - Construct only one instance ImageDataGenerator class for all types of augmenting

In [28]:
datagen_multiple = ImageDataGenerator(
    horizontal_flip=True, # Flip horizontally
    vertical_flip=True, # Flip vertically
    rotation_range=90, # Rotation_plus and minus
    height_shift_range=[-10,10], # Shift vertical
    width_shift_range=[-10,10], # Shift horizontal
    preprocessing_function = add_random_noise, # Random noise addition
    fill_mode='constant')

# 4/9 & 5/9 & 6/9 training datasets

In [29]:
# 4/9
# Generate batches of augmented images from this image
datagen_multiple_batch_1 = [datagen_multiple.flow(trainingImages_1[i]) for i in range(len(trainingImages_1))]
# Get 5 samples of each type of augmented images for each initial image
notFlatten_augmeneted_datagen_multiple_1 = [ [next(item)[0].astype(np.float32) for i in range(5)] for item in datagen_multiple_batch_1]
# Flatten lists
augmeneted_datagen_multiple_1 = [item for sublist in notFlatten_augmeneted_datagen_multiple_1 for item in sublist]

# 4/9 training dataset
finalImages_4 = []
finalLabels_4 = []

for i in range(len(trainingImages_1)):
    finalImages_4.append(cv2.resize(trainingImages_1[i][0,:,:,0], dsize=(128, 128), interpolation=cv2.INTER_CUBIC))
    finalLabels_4.append(trainingLabels_1[i])

for i in range(len(augmeneted_datagen_multiple_1)):
    
    # Resize image in 128x128 format
    finalImages_4.append(cv2.resize(augmeneted_datagen_multiple_1[i][:,:,0], dsize=(128, 128), interpolation=cv2.INTER_CUBIC))
    finalLabels_4.append(trainingLabels_1[int(i/5)])


# 5/9
# Generate batches of augmented images from this image
datagen_multiple_batch_2 = [datagen_multiple.flow(trainingImages_2[i]) for i in range(len(trainingImages_2))]
# Get 5 samples of each type of augmented images for each initial image
notFlatten_augmeneted_datagen_multiple_2 = [ [next(item)[0].astype(np.float32) for i in range(5)] for item in datagen_multiple_batch_2]
# Flatten lists
augmeneted_datagen_multiple_2 = [item for sublist in notFlatten_augmeneted_datagen_multiple_2 for item in sublist]
# 5/9 training dataset
finalImages_5 = []
finalLabels_5 = []

for i in range(len(trainingImages_2)):
    finalImages_5.append(cv2.resize(trainingImages_2[i][0,:,:,0], dsize=(128, 128), interpolation=cv2.INTER_CUBIC))
    finalLabels_5.append(trainingLabels_2[i])

for i in range(len(augmeneted_datagen_multiple_2)):
    
    # Resize image in 128x128 format
    finalImages_5.append(cv2.resize(augmeneted_datagen_multiple_2[i][:,:,0], dsize=(128, 128), interpolation=cv2.INTER_CUBIC))
    finalLabels_5.append(trainingLabels_2[int(i/5)])

# 6/9
# Generate batches of augmented images from this image
datagen_multiple_batch_3 = [datagen_multiple.flow(trainingImages_3[i]) for i in range(len(trainingImages_3))]
# Get 5 samples of each type of augmented images for each initial image
notFlatten_augmeneted_datagen_multiple_3 = [ [next(item)[0].astype(np.float32) for i in range(5)] for item in datagen_multiple_batch_3]
# Flatten lists
augmeneted_datagen_multiple_3 = [item for sublist in notFlatten_augmeneted_datagen_multiple_3 for item in sublist]

# 6/9 training dataset
finalImages_6 = []
finalLabels_6 = []

for i in range(len(trainingImages_3)):
    finalImages_6.append(cv2.resize(trainingImages_3[i][0,:,:,0], dsize=(128, 128), interpolation=cv2.INTER_CUBIC))
    finalLabels_6.append(trainingLabels_3[i])

for i in range(len(augmeneted_datagen_multiple_3)):
    
    # Resize image in 128x128 format
    finalImages_6.append(cv2.resize(augmeneted_datagen_multiple_3[i][:,:,0], dsize=(128, 128), interpolation=cv2.INTER_CUBIC))
    finalLabels_6.append(trainingLabels_3[int(i/5)])

# 13.3 Third type of generation - Construct only one instance ImageDataGenerator class for all types of augmenting (geometric operations)

In [30]:
datagen_geometric = ImageDataGenerator(
    horizontal_flip=True, # Flip horizontally
    vertical_flip=True, # Flip vertically
    rotation_range=90, # Rotation_plus and minus
    height_shift_range=[-10,10], # Shift vertical
    width_shift_range=[-10,10], # Shift horizontal
    fill_mode='constant')

# 7/9 & 8/9 & 9/9 training datasets

In [31]:
# 7/9
# Generate batches of augmented images from this image
datagen_geometric_batch_1 = [datagen_geometric.flow(trainingImages_1[i]) for i in range(len(trainingImages_1))]
# Get 5 samples of each type of augmented images for each initial image
notFlatten_augmeneted_datagen_geometric_1 = [ [next(item)[0].astype(np.float32) for i in range(5)] for item in datagen_geometric_batch_1]
# Flatten lists
augmeneted_datagen_geometric_1 = [item for sublist in notFlatten_augmeneted_datagen_geometric_1 for item in sublist]
# 7/9 training dataset
finalImages_7 = []
finalLabels_7 = []

for i in range(len(trainingImages_1)):
    finalImages_7.append(cv2.resize(trainingImages_1[i][0,:,:,0], dsize=(128, 128), interpolation=cv2.INTER_CUBIC))
    finalLabels_7.append(trainingLabels_1[i])

for i in range(len(augmeneted_datagen_geometric_1)):
    
    # Resize image in 128x128 format
    finalImages_7.append(cv2.resize(augmeneted_datagen_geometric_1[i][:,:,0], dsize=(128, 128), interpolation=cv2.INTER_CUBIC))
    finalLabels_7.append(trainingLabels_1[int(i/5)])

# 8/9
# Generate batches of augmented images from this image
datagen_geometric_batch_2 = [datagen_geometric.flow(trainingImages_2[i]) for i in range(len(trainingImages_2))]
# Get 5 samples of each type of augmented images for each initial image
notFlatten_augmeneted_datagen_geometric_2 = [ [next(item)[0].astype(np.float32) for i in range(5)] for item in datagen_geometric_batch_2]
# Flatten lists
augmeneted_datagen_geometric_2 = [item for sublist in notFlatten_augmeneted_datagen_geometric_2 for item in sublist]

# 8/9 training dataset
finalImages_8 = []
finalLabels_8 = []

for i in range(len(trainingImages_2)):
    finalImages_8.append(cv2.resize(trainingImages_2[i][0,:,:,0], dsize=(128, 128), interpolation=cv2.INTER_CUBIC))
    finalLabels_8.append(trainingLabels_2[i])

for i in range(len(augmeneted_datagen_geometric_2)):
    
    # Resize image in 128x128 format
    finalImages_8.append(cv2.resize(augmeneted_datagen_geometric_2[i][:,:,0], dsize=(128, 128), interpolation=cv2.INTER_CUBIC))
    finalLabels_8.append(trainingLabels_2[int(i/5)])

# 9/9
# Generate batches of augmented images from this image
datagen_geometric_batch_3 = [datagen_geometric.flow(trainingImages_3[i]) for i in range(len(trainingImages_3))]
# Get 5 samples of each type of augmented images for each initial image
notFlatten_augmeneted_datagen_geometric_3 = [ [next(item)[0].astype(np.float32) for i in range(5)] for item in datagen_geometric_batch_3]
# Flatten lists
augmeneted_datagen_geometric_3 = [item for sublist in notFlatten_augmeneted_datagen_geometric_3 for item in sublist]

# 9/9 training dataset
finalImages_9 = []
finalLabels_9 = []

for i in range(len(trainingImages_3)):
    finalImages_9.append(cv2.resize(trainingImages_3[i][0,:,:,0], dsize=(128, 128), interpolation=cv2.INTER_CUBIC))
    finalLabels_9.append(trainingLabels_3[i])

for i in range(len(augmeneted_datagen_geometric_3)):
    
    # Resize image in 128x128 format
    finalImages_9.append(cv2.resize(augmeneted_datagen_geometric_3[i][:,:,0], dsize=(128, 128), interpolation=cv2.INTER_CUBIC))
    finalLabels_9.append(trainingLabels_3[int(i/5)])

# III. PRE-PROCESSING PHASE

# 14. Normalization -  I normalize all images in order to have zero mean

In [22]:
def mean_normalization(image):
    # First, I only choose the non-zero intensity voxels - especially for MRI images beacause I don't want to normlize the brain area or another
    # organisms with the intensities of the pixels(voxels) around them
    mask = np.where(image != 0)

    # Then, I calculate the min and std (standard deviation)
    desired_img = image[mask]
    mean = np.mean(desired_img)
    std = np.std(desired_img)

    # And use them for normalization
    final_image = (image - mean)/std
    
    return final_image

In [23]:
# Normalize all training images from first final training dataset
normalized_images_1 = [mean_normalization(finalImages_1[i]) for i in range(len(finalImages_1))]

In [34]:
# Normalize all training datasets, except first dataset
normalized_images_2 = [mean_normalization(finalImages_2[i]) for i in range(len(finalImages_2))]
normalized_images_3 = [mean_normalization(finalImages_3[i]) for i in range(len(finalImages_3))]
normalized_images_4 = [mean_normalization(finalImages_4[i]) for i in range(len(finalImages_4))]
normalized_images_5 = [mean_normalization(finalImages_5[i]) for i in range(len(finalImages_5))]
normalized_images_6 = [mean_normalization(finalImages_6[i]) for i in range(len(finalImages_6))]
normalized_images_7 = [mean_normalization(finalImages_7[i]) for i in range(len(finalImages_7))]

# Normalized data visualization INTERFACE - interactive field for data visualization

In [ ]:
# I want to have an interactive view which can be controlled by this parameter:
# 18819 images

@interact
def explore_2d_cropped_images_training(
    image = widgets.IntSlider(
        min = 1,
        max = len(normalized_images_1),
        step = 1,
        description='Image: ',
        value = 10000)):
    
    index = image - 1
    array_view_1 = finalImages_1[index] # Use this array for visualization
    array_view_2 = normalized_images_1[index]
    
    # DImension of images
    dim1_1 = str(array_view_1.shape[0])
    dim1_2 = str(array_view_1.shape[1])
    dim2_1 = str(array_view_2.shape[0])
    dim2_2 = str(array_view_2.shape[1])
    
    # Intensity ranges for images
    intensity1_1 = str(round(array_view_1.min(), 1)) 
    intensity1_2 = str(round(array_view_1.max(), 1))
    intensity2_1 = str(round(array_view_2.min(), 1))
    intensity2_2 = str(round(array_view_2.max(), 1))

    tumor = 'YES' if finalLabels_1[index] == 1 else 'NO'

    pacient = str(int(index/36) + 1)
    layer = str(int(index%36) + 1)
    modal = str('FLAIR')
    view = str('axial')

    # Histograms
    img_before = finalImages_1[index]
    img_after = normalized_images_1[index]

    hist1_1, hist_centers1_1 = histogram(img_before, nbins=256)
    img_cdf1_1, bins1_1 = exposure.cumulative_distribution(img_before)

    hist2_1, hist_centers2_1 = histogram(img_after, nbins=256)
    img_cdf2_1, bins2_1 = exposure.cumulative_distribution(img_after)

    fig, ax = plt.subplots(2,2, figsize=(20,8.5)) # Set dimensions for view field
    ax[0,0].set_title('Before normalizing\n Image size: ' + dim1_1 + 'x' + dim1_2 + '\nIntensity: (' + intensity1_1 + ', ' + intensity1_2 + ')' +
                        '\nPacient: ' + pacient + '\nLayer: ' + layer + '\nModality: ' + modal + '\nView: ' + view + '\nTUMOR: ' + tumor) 
    ax[0,0].imshow(array_view_1, cmap = 'gray')
    ax[0,0].axis('off')
    ax[0,1].set_title('After normalizing\n Image size: ' + dim2_1 + 'x' + dim2_2 + '\nIntensity: (' + intensity2_1 + ', ' + intensity2_2 + ')' +
                        '\nPacient: ' + pacient + '\nLayer: ' + layer + '\nModality: ' + modal + '\nView: ' + view + '\nTUMOR: ' + tumor)
    ax[0,1].imshow(array_view_2, cmap = 'gray')
    ax[0,1].axis('off')

    ax[1,0].plot(bins1_1, img_cdf1_1, 'r')
    ax[1,0].plot(hist_centers1_1, hist1_1)
    
    ax[1,1].plot(bins2_1, img_cdf2_1, 'r')
    ax[1,1].plot(hist_centers2_1, hist2_1)

    # Zooming for much clearer plots beacause images have different scaling
    ax[1,0].axis([0.01, 200, 0, 5e2])
    ax[1,1].axis([-1.5, 3, 0, 5e2])

In [ ]:
print('Number of training instances is: ', len(finalImages_1))

# IV. CONVOLUTION NEURAL NETWORK

# IV.A - CLASSIFICATION TASK

# 15. Here is built the training dataset that will be passed to the CNN

In [26]:
finalImages_1 = np.array(finalImages_1)
finalLabels_1 = np.array(finalLabels_1)

# Shuffle data
randomIdx = np.random.permutation(len(finalImages_1))
images_1 = finalImages_1[randomIdx]
labels_1 = finalLabels_1[randomIdx]

# Dataset division
dataset_size_1 = len(images_1)
length_train_1 = int(0.7 * dataset_size_1)

train_images_1 = images_1[:length_train_1]
train_labels_1 = labels_1[:length_train_1]

test_images_1 = images_1[length_train_1:]
test_labels_1 = labels_1[length_train_1:]

# Save test images indexes
test_images_indexes = randomIdx[length_train_1:]

In [28]:
np.set_printoptions(threshold = np.inf)
# print(randomIdx)

new_array = np.array(randomIdx)

# Displaying the array

file = open("sample.txt", "w+")

# Saving the array in a text file
content = str(new_array)
file.write(content)
file.close()


# 16. Setting the values of the CNN parameters

In [29]:
EPOCHS_NO = 45 # Number of epochs
LR = 1.e-4 # Learning Rate value
BATCH_SIZE = 20 # Batch dimension

# 17. Build the network architecture

In [ ]:
# Convolutional model
SIZE = 128

INPUT_SHAPE = (SIZE, SIZE, 1) # change to (SIZE, SIZE, 1)

model_brain = Sequential()

model_brain.add(Conv2D(512, (3,3), input_shape = INPUT_SHAPE, padding='same')) # 512
model_brain.add(Activation('relu'))
model_brain.add(MaxPooling2D(pool_size=(2,2), strides = 2, padding='same'))

model_brain.add(Conv2D(256, (3,3), padding='same')) # 256
model_brain.add(Activation('relu'))

model_brain.add(Conv2D(128, (3,3), padding='same')) # 128
model_brain.add(Activation('relu'))
model_brain.add(MaxPooling2D(pool_size=(2,2), strides = 2, padding='same'))

model_brain.add(Conv2D(64, (3,3), padding='same')) # 64
model_brain.add(Activation('relu'))

model_brain.add(Conv2D(32, (3,3), padding='same')) # 32
model_brain.add(Activation('relu'))
model_brain.add(MaxPooling2D(pool_size=(2,2), strides = 2, padding='same'))

model_brain.add(Flatten()) # Flatten

model_brain.add(Dense(32))
model_brain.add(Activation('relu'))

model_brain.add(Dense(1))
model_brain.add(Activation('sigmoid'))

model_brain.compile(
    loss = tf.losses.binary_crossentropy,
    optimizer = tf.keras.optimizers.Adam(), 
    metrics=[tf.keras.metrics.BinaryAccuracy(name = 'accuracy')])
print(model_brain.summary())

# 18. Images to be processed by CNN must also contain in shape the number of color channels, respectively 1 for grayscale images

In [30]:
train_images_1 = [train_images_1[i].reshape(train_images_1[i].shape + (1, )) for i in range(len(train_images_1))]
test_images_1 = [test_images_1[i].reshape(test_images_1[i].shape + (1, )) for i in range(len(test_images_1))]

train_images_1 = np.array(train_images_1)
test_images_1 = np.array(test_images_1)

# 19. Due to a very high need for resources to train the CNN in a short time, GPU processing will be performed

In [ ]:
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

# 20. Perform CNN training

# 20.1 First model

In [42]:
# Configurations to save the network with the optimal values for weights, considering value of the loss function
checkpoint_filepath = 'D:\ATNN\BrainTumorDetector\SavedModels'
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=False,
    monitor='loss',
    mode='min',
    save_best_only=True)

In [ ]:
# Train model 1
initial_history = model_brain.fit(
    train_images_1, train_labels_1,
    batch_size = 20,
    epochs = EPOCHS_NO,
    validation_data = (test_images_1, test_labels_1),
    callbacks = [tf.keras.callbacks.LearningRateScheduler(lambda epoch: LR * (pow((1 - epoch/EPOCHS_NO), (0.9)))), model_checkpoint_callback,
    tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=3)]
)

# SAVE MODEL
# model_brain.save('last_model_brain_tumor_detection.h5') # Commented so as not to overwrite

# 20.2 Second model

In [43]:
# Configurations to save the network with the optimal values for weights, considering value of the loss function
checkpoint_filepath = 'D:\ATNN\BrainTumorDetector\SavedModels\Second'
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=False,
    monitor='loss',
    mode='min',
    save_best_only=True)

In [ ]:
# Train model 2
initial_history = model_brain.fit(
    train_images_1, train_labels_1,
    batch_size = 20,
    epochs = EPOCHS_NO,
    validation_data = (test_images_1, test_labels_1),
    callbacks = [tf.keras.callbacks.LearningRateScheduler(lambda epoch: LR * (pow((1 - epoch/EPOCHS_NO), (0.9)))), model_checkpoint_callback,
    tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=3)]
)

# SAVE MODEL
model_brain.save('second_model_brain_tumor_detection.h5') # Commented so as not to overwrite

In [ ]:
# construct a plot that plots and saves the training history
N = np.arange(0, 21)
plt.style.use("ggplot")
plt.figure()
plt.plot(N, initial_history.history["accuracy"], label="train_loss")
plt.plot(N, initial_history.history["val_accuracy"], label="val_loss")
plt.title("Training Accuracy")
plt.xlabel("Epoch #")
plt.ylabel("Accuracy")
plt.legend(loc="lower left")
plt.savefig('SecondModelTrainingAccuracy.png')

# 20.3 Third model

In [35]:
# Configurations to save the network with the optimal values for weights, considering value of the loss function
checkpoint_filepath = 'D:\ATNN\BrainTumorDetector\SavedModels\Third'
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=False,
    monitor='loss',
    mode='min',
    save_best_only=True)

In [ ]:
# Train model 3
initial_history = model_brain.fit(
    train_images_1, train_labels_1,
    batch_size = 20,
    epochs = EPOCHS_NO,
    validation_data = (test_images_1, test_labels_1),
    callbacks = [tf.keras.callbacks.LearningRateScheduler(lambda epoch: LR * (pow((1 - epoch/EPOCHS_NO), (0.9)))), model_checkpoint_callback,
    tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=3)]
)

# SAVE MODEL
model_brain.save('third_model_brain_tumor_detection.h5') # Commented so as not to overwrite

In [ ]:
# construct a plot that plots and saves the training history
N = np.arange(0, 25)
plt.style.use("ggplot")
plt.figure()
plt.plot(N, initial_history.history["accuracy"], label="train_loss")
plt.plot(N, initial_history.history["val_accuracy"], label="val_loss")
plt.title("Training Accuracy")
plt.xlabel("Epoch #")
plt.ylabel("Accuracy")
plt.legend(loc="lower left")
plt.savefig('ThirdModelTrainingAccuracy.png')

# 20.4 Fourth model

In [36]:
# Configurations to save the network with the optimal values for weights, considering value of the loss function
checkpoint_filepath = 'D:\ATNN\BrainTumorDetector\SavedModels\Fourth'
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=False,
    monitor='loss',
    mode='min',
    save_best_only=True)

In [ ]:
# Train model 4
initial_history = model_brain.fit(
    train_images_1, train_labels_1,
    batch_size = 20,
    epochs = EPOCHS_NO,
    validation_data = (test_images_1, test_labels_1),
    callbacks = [tf.keras.callbacks.LearningRateScheduler(lambda epoch: LR * (pow((1 - epoch/EPOCHS_NO), (0.9)))), model_checkpoint_callback,
    tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=3)]
)

# SAVE MODEL
model_brain.save('fourth_model_brain_tumor_detection.h5') # Commented so as not to overwrite

In [ ]:
# construct a plot that plots and saves the training history
N = np.arange(0, 13)
plt.style.use("ggplot")
plt.figure()
plt.plot(N, initial_history.history["accuracy"], label="train_loss")
plt.plot(N, initial_history.history["val_accuracy"], label="val_loss")
plt.title("Training Accuracy")
plt.xlabel("Epoch #")
plt.ylabel("Accuracy")
plt.legend(loc="lower left")
plt.savefig('FourthModelTrainingAccuracy.png')

In [ ]:
# construct a plot that plots and saves the training history
N = np.arange(0, 13)
plt.style.use("ggplot")
plt.figure()
plt.plot(N, initial_history.history["loss"], label="train_loss")
plt.plot(N, initial_history.history["val_loss"], label="val_loss")
plt.title("Training Loss")
plt.xlabel("Epoch #")
plt.ylabel("Loss")
plt.legend(loc="lower left")
plt.savefig('FourthModelTrainingLoss.png')

In [ ]:
# construct a plot that plots and saves the training history
N = np.arange(0, 13)
plt.style.use("ggplot")
plt.figure()
plt.plot(N, initial_history.history["roc"], label="train_loss")
plt.plot(N, initial_history.history["val_roc"], label="val_loss")
plt.title("Training Loss")
plt.xlabel("Epoch #")
plt.ylabel("Loss")
plt.legend(loc="lower left")
plt.savefig('FourthModelTrainingLoss.png')

In [32]:
# Model
myModel = tf.keras.models.load_model("fourth_model_brain_tumor_detection.h5")
y_pred = myModel.predict(test_images_1)
y_pred = [1 if y_pred[i] > 1.e-1 else 0 for i in range(len(test_images_1))]
y_test = test_labels_1

# Accuracy
acc = metrics.accuracy_score(y_test, y_pred)

In [ ]:
# AUC curve
fpr, tpr, _ = metrics.roc_curve(y_test,  y_pred)
auc = metrics.roc_auc_score(y_test, y_pred)

plt.plot(fpr, tpr, label = "ROC curve, AUC = " + str(round(auc, 2)))

leg = plt.legend(loc = 4)

#for text in leg.get_texts():
#    text.set_color("black")

plt.plot([0, 1], [0, 1],'b--')
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')

# plt.title('Receiver Operating Characteristic (ROC) curve')
plt.title(label="Receiver Operating Characteristic (ROC) curve",
          fontsize=12,
          color="black")
plt.savefig('FourthModelCurveROC_2.png')
plt.show()

# 21. Test trained model - Load trained model

In [32]:
myModel = tf.keras.models.load_model("fourth_model_brain_tumor_detection.h5")

In [ ]:
# Display test instance classification results
# Interactive field
# I want to have an interactive view which can be controlled by this parameter:
# 5646 images

test_images_1 = np.array(test_images_1)
@interact
def explore_final_test_classified_images(
    image = widgets.IntSlider(
        min = 1,
        max = len(test_images_1),
        step = 1,
        description='Image: ',
        value = 5000)):
    
    
    index = image - 1
    array_view = test_images_1[index] # Use this array for visualization

    
    dim1 = str(array_view.shape[0])
    dim2 = str(array_view.shape[1])
    
    
    intensity1 = str(round(array_view.min(), 1))
    intensity2 = str(round(array_view.max(), 1))

    pacient = str(int(index/51) + 1)
    augmented_instance = str(int(index%51) + 1)

    modal = str('FLAIR')
    view = str('axial')
    
    tumor = ''
    
    imagine_test = test_images_1[index]
    imagine_test = imagine_test.reshape((1, ) + imagine_test.shape)
    predicition = myModel.predict([imagine_test])
    
    
    if predicition[0][0] < 1.e-2:
        tumor = 'NO'
    else:
        tumor = 'YES'
    
    plt.figure(figsize=(5,5))

    plt.title(label='Explore images from dataset' + "\nImage size: " + dim1 + 'x' + dim2 + "\nIntensity: (" + intensity1 + ', ' + intensity2 + ')' +
    "\nPacient: " + pacient + "\nLayer: 90" + "\nInstance: " + augmented_instance + "\nModality: " + modal + "\nView: " + view + '\nTUMOR: ' + tumor,
          fontsize=12)
    
    plt.imshow(array_view, cmap = 'gray')

# 22. Create an array with classes for all test instaces

In [33]:
test_images_1 = [expand_dims(test_images_1[i], axis=0) for i in range(len(test_images_1))]

In [ ]:
classes = [myModel.predict([test_images_1[i]]) for i in range(len(test_images_1))]
final_classes = [ 1 if (classes[i])[0][0] > 1.e-2 else 0 for i in range(len(classes))]

# Select indexes of 'Tumor' (label == 1) instances
tumorIndexes = []
for i in range(len(final_classes)):
    if final_classes[i] == 1:
        tumorIndexes.append(i)

print(len(final_classes))

In [35]:
# Select from testing dataset only tumor instances
tumorInstances = [] # This array contains only images for which the network predicted class 1  
for i in range(len(tumorIndexes)):
    tumorInstances.append(test_images_1[tumorIndexes[i]])

# 23. Interface for viewing data detected as being in the 'tumor' category - interactive field for data visualization

In [ ]:
# Display test instance classification results
# Interactive field
# I want to have an interactive view which can be controlled by this parameter:
# 5646 images

tumorInstances = np.array(tumorInstances)
@interact
def explore_final_test_classified_images(
    image = widgets.IntSlider(
        min = 1,
        max = len(tumorInstances),
        step = 1,
        description='Image: ',
        value = 3000)):
    
    index = image - 1
    array_view = tumorInstances[index][0] # Use this array for visualization

    dim1 = str(array_view.shape[0])
    dim2 = str(array_view.shape[1])
    
    intensity1 = str(round(array_view.min(), 1))
    intensity2 = str(round(array_view.max(), 1))

    modal = str('FLAIR')
    view = str('axial')

    plt.figure(figsize=(5,5))

    plt.title(label='Explore images from dataset' + "\nImage size: " + dim1 + 'x' + dim2 + "\nIntensity: (" + intensity1 + ', ' + intensity2 + ')' +
    "\nLayer: 90" + "\nModality: " + modal + "\nView: " + view + '\nTUMOR: YES',
          fontsize=12,
          color="black")
    plt.imshow(array_view, cmap = 'gray')

# IV.B - SEGMENTATION TASK

# 24. Feature maps extraction

# 24.1 FEATURE MAPS display interface for images detected as 'tumor' - interactive field for data visualization

In [ ]:
# Display test instance classification results
# Interactive field
# I want to have an interactive view which can be controlled by this parameter:
# 4989 images

tumorInstances = np.array(tumorInstances)
@interact
def explore_final_test_classified_images(
    image = widgets.IntSlider(
        min = 1,
        max = len(tumorInstances),
        step = 1,
        description='Image: ',
        value = 3000)):
    
    index = image - 1
    # Image to verify
    array_view = tumorInstances[index] # Use this array for visualization
    
    # Show names of layers available in model
    layer_outputs = [layer.output for layer in myModel.layers[:]]
    activation_model = Model(inputs=myModel.input, outputs = layer_outputs)
    activations = activation_model.predict(array_view)
    fifth_conv_layer = activations[10]

    fig, ax = plt.subplots(4,8, figsize=(10,7))
    ax[0,0].imshow(fifth_conv_layer[0,:,:,0], cmap = 'gray')
    ax[0,0].axis('off')
    ax[0,1].imshow(fifth_conv_layer[0,:,:,1], cmap = 'gray')
    ax[0,1].axis('off')
    ax[0,2].imshow(fifth_conv_layer[0,:,:,2], cmap = 'gray')
    ax[0,2].axis('off')
    ax[0,3].imshow(fifth_conv_layer[0,:,:,3], cmap = 'gray')
    ax[0,3].axis('off')
    ax[0,4].imshow(fifth_conv_layer[0,:,:,4], cmap = 'gray')
    ax[0,4].axis('off')
    ax[0,5].imshow(fifth_conv_layer[0,:,:,5], cmap = 'gray')
    ax[0,5].axis('off')
    ax[0,6].imshow(fifth_conv_layer[0,:,:,6], cmap = 'gray')
    ax[0,6].axis('off')
    ax[0,7].imshow(fifth_conv_layer[0,:,:,7], cmap = 'gray')
    ax[0,7].axis('off')

    ax[1,0].imshow(fifth_conv_layer[0,:,:,8], cmap = 'gray')
    ax[1,0].axis('off')
    ax[1,1].imshow(fifth_conv_layer[0,:,:,9], cmap = 'gray')
    ax[1,1].axis('off')
    ax[1,2].imshow(fifth_conv_layer[0,:,:,10], cmap = 'gray')
    ax[1,2].axis('off')
    ax[1,3].imshow(fifth_conv_layer[0,:,:,11], cmap = 'gray')
    ax[1,3].axis('off')
    ax[1,4].imshow(fifth_conv_layer[0,:,:,12], cmap = 'gray')
    ax[1,4].axis('off')
    ax[1,5].imshow(fifth_conv_layer[0,:,:,13], cmap = 'gray')
    ax[1,5].axis('off')
    ax[1,6].imshow(fifth_conv_layer[0,:,:,14], cmap = 'gray')
    ax[1,6].axis('off')
    ax[1,7].imshow(fifth_conv_layer[0,:,:,15], cmap = 'gray')
    ax[1,7].axis('off')

    ax[2,0].imshow(fifth_conv_layer[0,:,:,16], cmap = 'gray')
    ax[2,0].axis('off')
    ax[2,1].imshow(fifth_conv_layer[0,:,:,17], cmap = 'gray')
    ax[2,1].axis('off')
    ax[2,2].imshow(fifth_conv_layer[0,:,:,18], cmap = 'gray')
    ax[2,2].axis('off')
    ax[2,3].imshow(fifth_conv_layer[0,:,:,19], cmap = 'gray')
    ax[2,3].axis('off')
    ax[2,4].imshow(fifth_conv_layer[0,:,:,20], cmap = 'gray')
    ax[2,4].axis('off')
    ax[2,5].imshow(fifth_conv_layer[0,:,:,21], cmap = 'gray')
    ax[2,5].axis('off')
    ax[2,6].imshow(fifth_conv_layer[0,:,:,22], cmap = 'gray')
    ax[2,6].axis('off')
    ax[2,7].imshow(fifth_conv_layer[0,:,:,23], cmap = 'gray')
    ax[2,7].axis('off')

    ax[3,0].imshow(fifth_conv_layer[0,:,:,24], cmap = 'gray')
    ax[3,0].axis('off')
    ax[3,1].imshow(fifth_conv_layer[0,:,:,25], cmap = 'gray')
    ax[3,1].axis('off')
    ax[3,2].imshow(fifth_conv_layer[0,:,:,26], cmap = 'gray')
    ax[3,2].axis('off')
    ax[3,3].imshow(fifth_conv_layer[0,:,:,27], cmap = 'gray')
    ax[3,3].axis('off')
    ax[3,4].imshow(fifth_conv_layer[0,:,:,28], cmap = 'gray')
    ax[3,4].axis('off')
    ax[3,5].imshow(fifth_conv_layer[0,:,:,29], cmap = 'gray')
    ax[3,5].axis('off')
    ax[3,6].imshow(fifth_conv_layer[0,:,:,30], cmap = 'gray')
    ax[3,6].axis('off')
    ax[3,7].imshow(fifth_conv_layer[0,:,:,31], cmap = 'gray')
    ax[3,7].axis('off')

    fig.suptitle('The 32 feature maps corresponding to each image in the TUMOR category\n Image processing may involve a small latency\n BE PATIENT!',
    x = 0.5, y = 1,fontsize=16, color = "black")

# 24.2 Create list with all Feature Maps (for all images with tumor)

In [37]:
featureMapsList = []

layer_outputs = [layer.output for layer in myModel.layers[:]]
activation_model = Model(inputs=myModel.input, outputs = layer_outputs)

for i in range(len(tumorInstances)):
    imagine_verificare = tumorInstances[i]
    activations = activation_model.predict(imagine_verificare)
    fifth_conv_layer = activations[10]
    
    for j in range(len(fifth_conv_layer[0])):
        featureMapsList.append(fifth_conv_layer[0,:,:,j])

featureMapsList = np.array(featureMapsList)

# 25. Gradient generating phase for each feature map

In [38]:
grad_arrays = []
for i in range(len(featureMapsList)):
    img = featureMapsList[i]

    sobelX = cv2.Sobel(img, cv2.CV_32F, 1, 0)
    sobelY = cv2.Sobel(img, cv2.CV_32F, 0, 1)

    sobelX = np.uint8(np.absolute(sobelX))
    sobelY = np.uint8(np.absolute(sobelY))
    
    sobelCombined = cv2.bitwise_or(sobelX, sobelY)

    grad_arrays.append(sobelCombined)

grad_arrays = np.array(grad_arrays)

# 26. Display feature maps and corresponding gradients

In [ ]:
# I want to have an interactive view which can be controlled by this parameter:
# 159648 images

@interact
def explore_2d_feature_and_gradient(
    image = widgets.IntSlider(
        min = 1,
        max = len(featureMapsList),
        step = 1,
        description='Image: ',
        value = 10000)):
    
    index = image - 1
    array_view_1 = featureMapsList[index] # Use this array for visualization
    array_view_2 = grad_arrays[index]
    
    # DImension of images
    dim1_1 = str(array_view_1.shape[0])
    dim1_2 = str(array_view_1.shape[1])
    dim2_1 = str(array_view_2.shape[0])
    dim2_2 = str(array_view_2.shape[1])
    
    # Intensity ranges for images
    intensity1_1 = str(round(array_view_1.min(), 1)) 
    intensity1_2 = str(round(array_view_1.max(), 1))
    intensity2_1 = str(round(array_view_2.min(), 1))
    intensity2_2 = str(round(array_view_2.max(), 1))

    fig, ax = plt.subplots(1,2, figsize=(6,6)) # Set dimensions for view field
    ax[0].set_title('Before cropping\n Image size: ' + dim1_1 + 'x' + dim1_2 + '\nIntensity: (' + intensity1_1 + ', ' + intensity1_2 + ')') 
    ax[0].imshow(array_view_1, cmap= 'magma_r')
    ax[0].axis('off')

    ax[1].set_title('After cropping\n Image size: ' + dim2_1 + 'x' + dim2_2 + '\nIntensity: (' + intensity2_1 + ', ' + intensity2_2 + ')')
    ax[1].imshow(array_view_2, cmap= 'magma_r')
    ax[1].axis('off')

# 27. Find the average and max values for each gradient

In [39]:
average_values = []
max_values = []
for i in range(len(grad_arrays)):
    average_values.append(np.mean(grad_arrays[i]))
    max_values.append(np.max(grad_arrays[i]))

average_values = np.array(average_values)
max_values = np.array(max_values)

# 28. Multiplying feature maps with corresponding gradients

In [40]:
modifiedFeatureMapsList = np.copy(featureMapsList)

for i in range(len(featureMapsList)):
    for x in range(32):
        for y in range(32):
            modifiedFeatureMapsList[i][x][y] *= average_values[i]
            modifiedFeatureMapsList[i][x][y] *= max_values[i]

# 29. Create an array that contains len(modifiedFeatureMapsList)/32 elements. Each element contains 32 images with 32x32 dimension.

In [41]:
arr2d_modifiedFeatureMapsList = np. reshape(modifiedFeatureMapsList, (int(len(modifiedFeatureMapsList)/32), 32, 32, 32))

# 30. The average of the 32 images is determined for each initial image in the test dataset

In [42]:
meanImages = []
for i in range(len(arr2d_modifiedFeatureMapsList)):
    processingImage = np.zeros((32, 32))
    processingImage = np.float32(processingImage)
    for j in range(32):
        processingImage = cv2.add(processingImage, arr2d_modifiedFeatureMapsList[i][j])
    meanImage = processingImage/32
    meanImages.append(meanImage)

# 31. Segmented image viewing interface - interactive field for data visualization

In [ ]:
# Display test instance classification results
# Interactive field
# I want to have an interactive view which can be controlled by this parameter:
# 4989 images

meanImages = np.array(meanImages)
@interact
def explore_final_test_classified_images(
    image = widgets.IntSlider(
        min = 1,
        max = len(meanImages),
        step = 1,
        description='Image: ',
        value = 3000)):
    
    index = image - 1
    array_view = meanImages[index] # Use this array for visualization

    dim1 = str(array_view.shape[0])
    dim2 = str(array_view.shape[1])
    
    intensity1 = str(round(array_view.min(), 1))
    intensity2 = str(round(array_view.max(), 1))

    modal = str('FLAIR')
    view = str('axial')

    plt.figure(figsize=(5,4))

    plt.title(label='Explore final images from dataset' + "\nImage size: " + dim1 + 'x' + dim2 + "\nIntensity: (" + intensity1 + ', ' + intensity2 + ')' +
    "\nLayer: 90" + "\nModality: " + modal + "\nView: " + view + '\nTUMOR: YES',
          fontsize=12,
          color="black")
    plt.imshow(array_view, cmap = 'gray')
    plt.axis('off')

# 32. Normalize images to bring them in range (0, 255)

In [43]:
finalNormalizedImages = [(cv2.normalize(meanImages[i], None, alpha = 0, beta = 255, norm_type = cv2.NORM_MINMAX, dtype = cv2.CV_32F)) for i in range(len(meanImages))]
for elem in finalNormalizedImages:
    for i in range(32):
        for j in range(32):
            if elem[i][j] < 0:
                elem[i][j] = 0

# 33. Reversing the values of the pixel intesities so that the highest intensities represent the segmented tumor

In [44]:
invertedValuesArray = []
for element in finalNormalizedImages:
    mask = np.full(element.shape,255)
    mod_img = mask - element
    mod_img = mod_img.astype(np.uint8)
    invertedValuesArray.append(mod_img)

# 34. Inverted segmented image viewing interface - interactive field for data visualization

In [ ]:
# Display test instance classification results
# Interactive field
# I want to have an interactive view which can be controlled by this parameter:
# 4989 images

invertedValuesArray = np.array(invertedValuesArray)
@interact
def explore_final_images(
    image = widgets.IntSlider(
        min = 1,
        max = len(invertedValuesArray),
        step = 1,
        description='Image: ',
        value = 1000)):
    
    index = image - 1
    array_view = invertedValuesArray[index] # Use this array for visualization

    dim1 = str(array_view.shape[0])
    dim2 = str(array_view.shape[1])
    
    intensity1 = str(round(array_view.min(), 1))
    intensity2 = str(round(array_view.max(), 1))

    modal = str('FLAIR')
    view = str('axial')

    plt.figure(figsize=(5,4))

    plt.title(label='Explore final images from dataset' + "\nImage size: " + dim1 + 'x' + dim2 + "\nIntensity: (" + intensity1 + ', ' + intensity2 + ')' +
    "\nModality: " + modal + "\nView: " + view + '\nTUMOR: YES',
          fontsize=12,
          color="black")
    plt.imshow(array_view, cmap = 'gray')
    plt.axis('off')

# 35. Define a function to calculate the value of the similarity coefficient Dice

In [45]:
def compute_dice_coefficient(mask_gt, mask_pred):
  '''
  Computes soerensen-dice coefficient.

  compute the soerensen-dice coefficient between the ground truth mask `mask_gt`
  and the predicted mask `mask_pred`.

  Args:
    mask_gt: 3-dim Numpy array of type bool. The ground truth mask.
    mask_pred: 3-dim Numpy array of type bool. The predicted mask.

  Returns:
    the dice coeffcient as float. If both masks are empty, the result is NaN.
  '''
  volume_sum = mask_gt.sum() + mask_pred.sum()
  if volume_sum == 0:
    return np.NaN
  volume_intersect = (mask_gt & mask_pred).sum()
  return 2 * volume_intersect / volume_sum

# 36. Apply the treshold function to all images in the test dataset

In [46]:
width_dim = 152
height_dim = 192
dimension = (width_dim, height_dim)
resized_segmented = [(cv2.resize(invertedValuesArray[i], dimension, interpolation=cv2.INTER_LINEAR)) for i in
                            range(len(invertedValuesArray))]

In [47]:
tresholdResults = []
for i in range(len(invertedValuesArray)):
    ret, image_result = cv2.threshold(resized_segmented[i], np.max(resized_segmented[i])*0.75, np.max(resized_segmented[i]), cv2.THRESH_BINARY)
    tresholdResults.append(image_result)

# 38. Apply morphological operations and experimentally calculate the value of the similarity coefficient Dice

In [48]:
kernel = np.ones((3,3), np.uint8)
openedImages = [(cv2.morphologyEx(tresholdResults[i], cv2.MORPH_OPEN, kernel)) for i in range(len(tresholdResults))]

kernel2nd = np.ones((1, 1), np.float32)
openedImages2nd = [(cv2.morphologyEx(openedImages[i], cv2.MORPH_OPEN, kernel2nd)) for i in range(len(openedImages))]

blurredImages = [(cv2.medianBlur(openedImages2nd[i], 5)) for i in range(len(openedImages2nd))]

# Resize original image
invertedValuesArray = [(cv2.resize(invertedValuesArray[i], dimension, interpolation=cv2.INTER_LINEAR)) for i in
                           range(len(invertedValuesArray))]

In [ ]:
# Select only slice 90 of each image from trainingSegmented_beforeCropping
width = 128
height = 128
dim = (width, height)

only_90 = []
for i in range(int(len(trainingSegmented_beforeCropping)/155)):
    resized_img = cv2.resize(trainingSegmented_beforeCropping[int(89 + int(i*155))], dim, interpolation = cv2.INTER_LINEAR)
    ret, tresh = cv2.threshold(resized_img, 0, 255, cv2.THRESH_BINARY)
    only_90.append(tresh)
print(len(only_90))

new_array_90 = np.array(only_90)

In [ ]:

# Resize images
'''
width = 128
height = 128
dim = (width, height)
resized_segmented = [(cv2.resize(openedImages[i], dim, interpolation = cv2.INTER_LINEAR))  for i in range(len(openedImages))]

test_image = resized_segmented[2]

cropped_trainingSegmented = crop_images(trainingSegmented_beforeCropping[29695])
resized_original = cv2.resize(cropped_trainingSegmented, dim, interpolation = cv2.INTER_LINEAR)

res, original_image_result = cv2.threshold(resized_original, 0, 255, cv2.THRESH_BINARY)

dice_result_24 = compute_dice_coefficient(original_image_result, test_image)
width = 128
height = 128
dim = (width, height)
invertedValuesArray = [(cv2.resize(invertedValuesArray[i], dim, interpolation = cv2.INTER_LINEAR))  for i in range(len(invertedValuesArray))]
'''


# 39. Before and after segmenting image viewing interface

In [ ]:
# I want to have an interactive view which can be controlled by this parameter:
# 4989 images

@interact
def explore_segmented_images(
    image = widgets.IntSlider(
        min = 1,
        max = len(resized_segmented),
        step = 1,
        description='Image: ',
        value = 18)):
    
    index = image - 1
    array_view_1 = invertedValuesArray[index] # Use this array for visualization
    array_view_2 = resized_segmented[index]
    
    # DImension of images
    dim1_1 = str(array_view_1.shape[0])
    dim1_2 = str(array_view_1.shape[1])
    dim2_1 = str(array_view_2.shape[0])
    dim2_2 = str(array_view_2.shape[1])
    
    # Intensity ranges for images
    intensity1_1 = str(round(array_view_1.min(), 1)) 
    intensity1_2 = str(round(array_view_1.max(), 1))
    intensity2_1 = str(round(array_view_2.min(), 1))
    intensity2_2 = str(round(array_view_2.max(), 1))

    fig, ax = plt.subplots(1,2, figsize=(6,6)) # Set dimensions for view field
    ax[0].set_title('Before segmenting\n Image size: ' + dim1_1 + 'x' + dim1_2 + '\nIntensity: (' + intensity1_1 + ', ' + intensity1_2 + ')') 
    ax[0].imshow(array_view_1, cmap= 'gray')
    ax[0].axis('off')

    ax[1].set_title('After segmenting\n Image size: ' + dim2_1 + 'x' + dim2_2 + '\nIntensity: (' + intensity2_1 + ', ' + intensity2_2 + ')')
    ax[1].imshow(array_view_2, cmap= 'gray')
    ax[1].axis('off')

# 40. Create the superimposed images

In [49]:
# Create a mask with 0 intensity pixels for all elements from resized_segmented array
masks_null = [(np.where(blurredImages[i] == 0)) for i in range(len(blurredImages))]

# Apply a color map
mapped_colors = [(cv2.applyColorMap(invertedValuesArray[i], cv2.COLORMAP_RAINBOW)) for i in range(len(invertedValuesArray))]

In [ ]:
plt.imshow(mapped_colors[5])

In [ ]:
mapped_colors = np.array(mapped_colors)
for i in range(len(mapped_colors)):
    (mapped_colors[i])[masks_null[i]] = 0

In [ ]:
width = 152
height = 192
dim = (width, height)

original_images = [tumorInstances[i][0] for i in range(len(tumorInstances))]
original_images = np.array(original_images)
resized_original_images = [(cv2.resize(original_images[i], dim, interpolation = cv2.INTER_LINEAR)) for i in range(len(original_images))]
converted_images = [(np.float32(resized_original_images[i])) for i in range(len(resized_original_images))]
backtorgbs = [(cv2.cvtColor(converted_images[i], cv2.COLOR_GRAY2RGB)) for i in range(len(converted_images))]
backtorgbs = [(np.uint8(backtorgbs[i])) for i in range(len(backtorgbs))]
dst = [(cv2.add(backtorgbs[i], mapped_colors[i])) for i in range(len(backtorgbs))]

# 41. Display FINAL IMAGES

In [ ]:
@interact
def explore_2d_results(
    image = widgets.IntSlider(
        min = 1,
        max = len(dst),
        step = 1,
        description='Image: ',
        value = 100)):
    
    index = image - 1
    array_view_1 = original_images[index] # Use this array for visualization
    array_view_2 = dst[index]
    
    # DImension of images
    dim1_1 = str(array_view_1.shape[0])
    dim1_2 = str(array_view_1.shape[1])
    dim2_1 = str(array_view_2.shape[0])
    dim2_2 = str(array_view_2.shape[1])
    
    # Intensity ranges for images
    intensity1_1 = str(round(array_view_1.min(), 1)) 
    intensity1_2 = str(round(array_view_1.max(), 1))
    intensity2_1 = str(round(array_view_2.min(), 1))
    intensity2_2 = str(round(array_view_2.max(), 1))

    fig, ax = plt.subplots(1,2, figsize=(6,6)) # Set dimensions for view field
    ax[0].set_title('Before cropping\n Image size: ' + dim1_1 + 'x' + dim1_2 + '\nIntensity: (' + intensity1_1 + ', ' + intensity1_2 + ')') 
    ax[0].imshow(array_view_1, cmap = 'gray')
    ax[0].axis('off')

    ax[1].set_title('After cropping\n Image size: ' + dim2_1 + 'x' + dim2_2 + '\nIntensity: (' + intensity2_1 + ', ' + intensity2_2 + ')')
    ax[1].imshow(array_view_2)
    ax[1].axis('off')